# This notebook contain working on Rajshahi dataset Feature engineering and Model development Analysis using
### - Timeseries


In [1]:
import tensorflow as tf
print(tf.__version__)

2.13.1


## Import liabraries

In [2]:
import datetime

import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import Model, Sequential

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError

from tensorflow.keras.layers import Dense, Conv1D, LSTM, Lambda, Reshape, RNN, LSTMCell

import warnings
warnings.filterwarnings('ignore')

In [3]:
plt.rcParams['figure.figsize'] = (10, 7.5)
plt.rcParams['axes.grid'] = False

In [4]:
tf.random.set_seed(42)
np.random.seed(42)

In [5]:
train_df = pd.read_csv('../data/train.csv', index_col='datetime',parse_dates=True)
val_df = pd.read_csv('../data/val.csv', index_col='datetime',parse_dates=True)
test_df = pd.read_csv('../data/test.csv', index_col='datetime',parse_dates=True)

In [6]:
print(train_df.shape, val_df.shape, test_df.shape)

(2972, 25) (850, 25) (425, 25)


In [7]:
class DataWindow():
    def __init__(self, input_width, label_width, shift,
                 train_df=train_df, val_df=val_df, test_df=test_df,
                 label_columns=None):

        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in enumerate(label_columns)}
        self.column_indices = {name: i for i, name in enumerate(train_df.columns)}

        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def split_to_inputs_labels(self, features):
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        if self.label_columns is not None:
            labels = tf.stack(
                [labels[:,:,self.column_indices[name]] for name in self.label_columns],
                axis=-1
            )
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])

        return inputs, labels


    def make_dataset(self, data):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.preprocessing.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=self.total_window_size,
            sequence_stride=1,
            shuffle=True,
            batch_size=32
        )

        ds = ds.map(self.split_to_inputs_labels)
        return ds

    @property
    def train(self):
        return self.make_dataset(self.train_df)

    @property
    def val(self):
        return self.make_dataset(self.val_df)

    @property
    def test(self):
        return self.make_dataset(self.test_df)

    @property
    def sample_batch(self):
        result = getattr(self, '_sample_batch', None)
        if result is None:
            result = next(iter(self.train))
            self._sample_batch = result
            
        return result

In [8]:
class Baseline(Model):
    def __init__(self, label_index=None):
        super().__init__()
        self.label_index = label_index

    def call(self, inputs):
        if self.label_index is None:
            return inputs

        elif isinstance(self.label_index, list):
            tensors = []
            for index in self.label_index:
                result = inputs[:, :, index]
                result = result[:, :, tf.newaxis]
                tensors.append(result)
            return tf.concat(tensors, axis=-1)

        result = inputs[:, :, self.label_index]
        return result[:,:,tf.newaxis]

In [9]:
mo_single_step_window = DataWindow(input_width=1, label_width=1, shift=1, label_columns=['precip','rain_sum ','river_discharge'])
mo_wide_window = DataWindow(input_width=14, label_width=14, shift=1, label_columns=['precip','rain_sum ','river_discharge'])

In [10]:
column_indices = {name: i for i, name in enumerate(train_df.columns)}

In [11]:
print(column_indices['precip'])
print(column_indices['rain_sum '])
print(column_indices['river_discharge'])

8
22
19


In [13]:
mo_baseline_last = Baseline(label_index=[21,22,23])

mo_baseline_last.compile(loss=MeanSquaredError(), metrics=[MeanAbsoluteError()])

mo_val_performance = {}
mo_performance = {}

mo_val_performance['Baseline - Last'] = mo_baseline_last.evaluate(mo_wide_window.val)
mo_performance['Baseline - Last'] = mo_baseline_last.evaluate(mo_wide_window.test, verbose=0)

27/27 [==============================] - 0s 2ms/step - loss: 0.0227 - mean_absolute_error: 0.0740


In [14]:
print(mo_performance['Baseline - Last'][1])

0.10904384404420853


In [15]:
def compile_and_fit(model, window, patience=3, max_epochs=50):
    early_stopping = EarlyStopping(monitor='val_loss',
                                   patience=patience,
                                   mode='min')

    model.compile(loss=MeanSquaredError(),
                  optimizer=Adam(),
                  metrics=[MeanAbsoluteError()])

    history = model.fit(window.train,
                       epochs=max_epochs,
                       validation_data=window.val,
                       callbacks=[early_stopping])

    return history

In [16]:
mo_dense = Sequential([
    Dense(units=64, activation='relu'),
    Dense(units=64, activation='relu'),
    Dense(units=3)
])

history = compile_and_fit(mo_dense, mo_single_step_window)

mo_val_performance['Dense'] = mo_dense.evaluate(mo_single_step_window.val)
mo_performance['Dense'] = mo_dense.evaluate(mo_single_step_window.test, verbose=0)

Epoch 1/50
93/93 [==============================] - 1s 6ms/step - loss: 0.0104 - mean_absolute_error: 0.0595 - val_loss: 0.0054 - val_mean_absolute_error: 0.0467
Epoch 2/50
93/93 [==============================] - 0s 3ms/step - loss: 0.0024 - mean_absolute_error: 0.0283 - val_loss: 0.0034 - val_mean_absolute_error: 0.0364
Epoch 3/50
93/93 [==============================] - 0s 3ms/step - loss: 0.0020 - mean_absolute_error: 0.0237 - val_loss: 0.0024 - val_mean_absolute_error: 0.0289
Epoch 4/50
93/93 [==============================] - 0s 3ms/step - loss: 0.0020 - mean_absolute_error: 0.0241 - val_loss: 0.0018 - val_mean_absolute_error: 0.0267
Epoch 5/50
93/93 [==============================] - 0s 3ms/step - loss: 0.0019 - mean_absolute_error: 0.0218 - val_loss: 0.0014 - val_mean_absolute_error: 0.0241
Epoch 6/50
93/93 [==============================] - 0s 3ms/step - loss: 0.0018 - mean_absolute_error: 0.0208 - val_loss: 0.0013 - val_mean_absolute_error: 0.0219
Epoch 7/50
93/93 [==========

In [17]:
mo_lstm_model = Sequential([
    LSTM(32, return_sequences=True),
    Dense(units = 3)
])

history = compile_and_fit(mo_lstm_model, mo_wide_window)

mo_val_performance = {}
mo_performance = {}

mo_val_performance['LSTM'] = mo_lstm_model.evaluate(mo_wide_window.val)
mo_performance['LSTM'] = mo_lstm_model.evaluate(mo_wide_window.test, verbose=0)

Epoch 1/50
93/93 [==============================] - 4s 14ms/step - loss: 0.0160 - mean_absolute_error: 0.0782 - val_loss: 0.0056 - val_mean_absolute_error: 0.0514
Epoch 2/50
93/93 [==============================] - 1s 8ms/step - loss: 0.0051 - mean_absolute_error: 0.0420 - val_loss: 0.0035 - val_mean_absolute_error: 0.0383
Epoch 3/50
93/93 [==============================] - 1s 8ms/step - loss: 0.0035 - mean_absolute_error: 0.0324 - val_loss: 0.0025 - val_mean_absolute_error: 0.0314
Epoch 4/50
93/93 [==============================] - 1s 9ms/step - loss: 0.0028 - mean_absolute_error: 0.0277 - val_loss: 0.0020 - val_mean_absolute_error: 0.0277
Epoch 5/50
93/93 [==============================] - 1s 9ms/step - loss: 0.0024 - mean_absolute_error: 0.0247 - val_loss: 0.0016 - val_mean_absolute_error: 0.0253
Epoch 6/50
93/93 [==============================] - 1s 8ms/step - loss: 0.0022 - mean_absolute_error: 0.0234 - val_loss: 0.0014 - val_mean_absolute_error: 0.0234
Epoch 7/50
93/93 [=========

In [18]:
predicted_results = mo_baseline_last.predict(mo_wide_window.test)
predicted_array= predicted_results[0]

my_array = np.array(predicted_array)

df_raw = pd.DataFrame(my_array)

df = df_raw.rename(columns={0: "river_discharge", 1: "rain_sum ",2:"precip"})


df.head(14)

13/13 [==============================] - 0s 1ms/step


,river_discharge,rain_sum,precip
0,0.154309,0.154309,0.666667
1,0.034068,0.034068,0.416667
2,0.026052,0.026052,0.333333
3,0.030060,0.030060,0.250000
4,0.146293,0.146293,0.625000
5,0.326653,0.326653,0.666667
6,0.086172,0.086172,0.625000
7,0.098196,0.098196,0.500000
8,0.012024,0.012024,0.250000
9,0.012024,0.012024,0.208333


In [21]:
mo_lstm_model.save("lstm_rajshahi_model.h5")